In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pysal as ps
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
# from fancyimpute import KNN
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
%matplotlib inline
%config IPCompleter.greedy=True

/home/steve/anaconda3/lib/python3.7/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


In [2]:
df = pd.read_csv('fent_county_year.txt', sep = '\t')
fent_all_years = pd.read_csv('all_fentanyl_by_county.txt', sep = '\t')
fent_13_17 = pd.read_csv('all_fent_by_county_13-17.txt', sep = '\t')
national_fent = pd.read_csv('national_fent.txt', sep = '\t')
national_all_opioids = pd.read_csv('grouped_by_year_not_county_all_opioids.txt', sep = '\t')
national_fent_cocaine = pd.read_csv('fent_and_cocaine.txt', sep = '\t')
fent_by_state = pd.read_csv('fent_state.txt', sep = '\t')

In [3]:
df = df.drop('Notes', axis = 'columns')
df = df.dropna()
df = df.drop('Year Code', axis = 'columns')
df.columns = ['county', 'county_code', 'year', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_all_years = fent_all_years.drop('Notes', axis = 'columns')
fent_all_years = fent_all_years.dropna()
fent_all_years.columns = ['county', 'county_code', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_13_17 = fent_13_17.drop('Notes', axis = 'columns')
fent_13_17 = fent_13_17.dropna()
fent_13_17.columns = ['county', 'county_code', 'deaths', 'population', 'crude_rate', 'aa_rate']

national_fent = national_fent.drop('Notes', axis = 'columns')
national_fent = national_fent.dropna()
national_fent = national_fent.drop('Year Code', axis = 'columns')
national_fent.columns = ['year', 'deaths', 'population', 'crude_rate', 'aa_rate']

national_all_opioids = national_all_opioids.drop('Notes', axis = 'columns')
national_all_opioids = national_all_opioids.dropna()
national_all_opioids = national_all_opioids.drop('Year Code', axis = 'columns')
national_all_opioids.columns = ['year', 'deaths', 'population', 'crude_rate']

national_fent_cocaine = national_fent_cocaine.drop('Notes', axis = 'columns')
national_fent_cocaine = national_fent_cocaine.dropna()
national_fent_cocaine = national_fent_cocaine.drop('Year Code', axis = 'columns')
national_fent_cocaine.columns = ['year', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_by_state = fent_by_state.drop(['Notes', 'Year Code'], axis = 'columns')
fent_by_state = fent_by_state.dropna()
fent_by_state.columns = ['state', 'state_code', 'year', 'deaths', 'population', 'crude_rate', 'aa_rate']

In [4]:
# dropping hawaii and alaska because they make the map ugly
df['state'] = df.county.apply(lambda x: x[-2:])
df = df[df['state'] != 'AK']
df = df[df['state'] != 'HI']
fent_13_17['state'] = fent_13_17['county'].apply(lambda x: x[-2:])
fent_13_17 = fent_13_17[fent_13_17['state'] != 'AK']
fent_13_17 = fent_13_17[fent_13_17['state'] != 'HI']
fent_by_state_13on = fent_by_state[fent_by_state.year > 2012]
fent_by_state_13on = fent_by_state_13on[fent_by_state_13on.state != 'Alaska']
fent_by_state_13on = fent_by_state_13on[fent_by_state_13on.state != 'Hawaii']

In [5]:
df = df.drop(df[df['deaths'] == 'Missing'].index)
df = df.drop(df[df.county == 'Bedford city, VA'].index)

In [6]:
hdf = df[df.year == 2017][['county', 'county_code', 'state']]
hdf['deaths13'] = list(df[df.year == 2013].deaths)
hdf['deaths14'] = list(df[df.year == 2014].deaths)
hdf['deaths15'] = list(df[df.year == 2015].deaths)
hdf['deaths16'] = list(df[df.year == 2016].deaths)
hdf['deaths17'] = list(df[df.year == 2017].deaths)
hdf['pop13'] = list(df[df.year == 2013].population)
hdf['pop14'] = list(df[df.year == 2014].population)
hdf['pop15'] = list(df[df.year == 2015].population)
hdf['pop16'] = list(df[df.year == 2016].population)
hdf['pop17'] = list(df[df.year == 2017].population)
hdf.reset_index(drop = True, inplace = True)

In [7]:
hdf['total_deaths'] = hdf.merge(fent_13_17[['county', 'deaths']], on = 'county', how = 'left').deaths

In [8]:
state_abbr_dict = {'Alabama':'AL',
                    'Alaska':'AK',
                    'Arizona':'AZ',
                    'Arkansas':'AR',
                    'California': 'CA',
                    'Colorado': 'CO',
                    'Connecticut': 'CT',
                    'Delaware': 'DE',
                    'Florida': 'FL',
                    'Georgia': 'GA',
                    'Hawaii': 'HI',
                    'Idaho': 'ID',
                    'Illinois': 'IL',
                    'Indiana': 'IN',
                    'Iowa': 'IA',
                    'Kansas': 'KS',
                    'Kentucky': 'KY',
                    'Louisiana': 'LA',
                    'Maine': 'ME',
                    'Maryland': 'MD',
                    'Massachusetts': 'MA',
                    'Michigan': 'MI',
                    'Minnesota': 'MN',
                    'Mississippi': 'MS',
                    'Missouri': 'MO',
                    'Montana': 'MT',
                    'Nebraska': 'NE',
                    'Nevada': 'NV',
                    'New Hampshire': 'NH',
                    'New Jersey': 'NJ',
                    'New Mexico': 'NM',
                    'New York': 'NY',
                    'North Carolina': 'NC',
                    'North Dakota': 'ND',
                    'Ohio': 'OH',
                    'Oklahoma': 'OK',
                    'Oregon': 'OR',
                    'Pennsylvania': 'PA',
                    'Rhode Island': 'RI',
                    'South Carolina': 'SC',
                    'South Dakota': 'SD',
                    'Tennessee': 'TN',
                    'Texas': 'TX',
                    'Utah': 'UT',
                    'Vermont': 'VT',
                    'Virginia': 'VA',
                    'Washington': 'WA',
                    'West Virginia': 'WV',
                    'Wisconsin': 'WI',
                    'Wyoming': 'WY',
                    'District of Columbia': 'DC'}
fent_by_state['state_abbr'] = fent_by_state.state.map(state_abbr_dict)

In [9]:
counties = gpd.read_file('us-counties.json')
counties.columns = ['county_code', 'name', 'geometry']
counties.county_code = counties.county_code.apply(int)
hdf['county_code'] = hdf.county_code.apply(int)
hdf = counties[['county_code', 'geometry']].merge(hdf, on = 'county_code', how = 'right')

In [10]:
drop_county_codes = hdf[hdf.geometry.isna()].county_code
drop_index = hdf[hdf.geometry.isna()].index
hdf = hdf.drop(drop_index)

In [11]:
hdf.to_file(drive = 'ESRI Shapefile', filename = 'hdf.shp')
w = ps.queen_from_shapefile('hdf.shp')
# row standardization 
w.transform = 'r'

/home/steve/anaconda3/lib/python3.7/site-packages/pysal/core/IOHandlers/pyShpIO.py:172: RuntimeWarning: SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)
  warn("SHAPEFILE WARNING: Polygon %d topology has been fixed. (ccw -> cw)" % (self.pos), RuntimeWarning)


SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)


/home/steve/anaconda3/lib/python3.7/site-packages/pysal/core/IOHandlers/pyShpIO.py:172: RuntimeWarning: SHAPEFILE WARNING: Polygon 2881 topology has been fixed. (ccw -> cw)
  warn("SHAPEFILE WARNING: Polygon %d topology has been fixed. (ccw -> cw)" % (self.pos), RuntimeWarning)


SHAPEFILE WARNING: Polygon 2881 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2881 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2881 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2881 topology has been fixed. (ccw -> cw)


In [12]:
rent_burden = pd.read_csv('ACS_17_5YR_B25070_with_ann.csv')
rent_burden = rent_burden[['GEO.id2', 'GEO.display-label', 'HD01_VD01', 'HD01_VD10']]
rent_burden.columns = ['county_code', 'county', 'total_renters', 'burden_over_50_percent']
rent_burden['percent_in_distress'] = rent_burden['burden_over_50_percent']/rent_burden['total_renters']
rent_burden = rent_burden[['county_code', 'percent_in_distress']]
hdf = hdf.merge(rent_burden, on = 'county_code', how = 'left')

In [13]:
prescriptions_17 = pd.read_csv('2017_opioid_prescriptions.csv')[['State/County FIPS Code', '2017']]
prescriptions_16 = pd.read_csv('2016_opioid_prescriptions.csv')[['FIPS County Code', '2016 Prescribing Rate']]
prescriptions_15 = pd.read_csv('2015_opioid_prescriptions.csv')[['FIPS County Code', '2015 Prescribing Rate']]
prescriptions_14 = pd.read_csv('2014_opioid_prescriptions.csv')[['FIPS County Code', '2014 Prescribing Rate']]
prescriptions_13 = pd.read_csv('2013_opioid_prescriptions.csv')[['FIPS County Code', '2013 Prescribing Rate']]
prescriptions_17.columns = ['county_code', 'prate_17']
prescriptions_16.columns = ['county_code', 'prate_16']
prescriptions_15.columns = ['county_code', 'prate_15']
prescriptions_14.columns = ['county_code', 'prate_14']
prescriptions_13.columns = ['county_code', 'prate_13']
hdf = hdf.merge(prescriptions_13, on = 'county_code', how = 'left')
hdf = hdf.merge(prescriptions_14, on = 'county_code', how = 'left')
hdf = hdf.merge(prescriptions_15, on = 'county_code', how = 'left')
hdf = hdf.merge(prescriptions_16, on = 'county_code', how = 'left')
hdf = hdf.merge(prescriptions_17, on = 'county_code', how = 'left')

In [14]:
# converting to floats, turning non numbers to Nan
def convert_prate(value):
    try:
        return float(value)
    except ValueError:
        return None
hdf['prate_17'] = hdf.prate_17.apply(convert_prate)
hdf['prate_16'] = hdf.prate_16.apply(convert_prate)
hdf['prate_15'] = hdf.prate_15.apply(convert_prate)
hdf['prate_14'] = hdf.prate_14.apply(convert_prate)
hdf['prate_13'] = hdf.prate_13.apply(convert_prate)

In [15]:
def convert_suppressed(entry):
    if entry == 'Suppressed':
        return None
    else:
        return int(entry)
hdf['deaths13'] = hdf.deaths13.apply(convert_suppressed)
hdf['deaths14'] = hdf.deaths14.apply(convert_suppressed)
hdf['deaths15'] = hdf.deaths15.apply(convert_suppressed)
hdf['deaths16'] = hdf.deaths16.apply(convert_suppressed)
hdf['deaths17'] = hdf.deaths17.apply(convert_suppressed)
hdf['total_deaths'] = hdf.total_deaths.apply(convert_suppressed)

In [16]:
hdf['total_deaths'] = hdf.total_deaths.fillna(0)

In [17]:
fent_by_state['deaths'] = fent_by_state.deaths.apply(convert_suppressed)

In [18]:
fent_by_state = fent_by_state.fillna(0)

In [19]:
ratio_dict = {}
for state in hdf.state.unique():
    ratio_dict[state] = {}
    for year in range(2013, 2018):
        ratio_dict[state][year] = fent_by_state[(fent_by_state['state_abbr'] == state) & (fent_by_state['year'] == year)].deaths.sum() / fent_by_state[(fent_by_state['state_abbr'] == state) & (fent_by_state['year'] > 2012)].deaths.sum()

In [20]:
# I'm going to say I can't do better than 60983, the missing 5125 deaths I won't worry about
print('Total deaths 2013-2017: ', national_fent[national_fent.year > 2012].deaths.sum())
print('Deaths accounted for in county data:', hdf.total_deaths.sum())
print('Deaths I wont worry about: ', national_fent[national_fent.year > 2012].deaths.sum() - hdf.total_deaths.sum())

Total deaths 2013-2017:  66108.0
Deaths accounted for in county data: 60983.0
Deaths I wont worry about:  5125.0


In [21]:
def fill_missing(row, column, year):
    if row[column] > 0:
        return row[column]
    if row.total_deaths == 0:
        return 0
    else:
        pred_value = round(ratio_dict[row.state][year] * row.total_deaths)
        if pred_value >= 10:
            pred_value = 9
        return pred_value
hdf['ratio_predicted13'] = hdf.apply(lambda x: fill_missing(x, 'deaths13', 2013), axis = 'columns')
hdf['ratio_predicted14'] = hdf.apply(lambda x: fill_missing(x, 'deaths14', 2014), axis = 'columns')
hdf['ratio_predicted15'] = hdf.apply(lambda x: fill_missing(x, 'deaths15', 2015), axis = 'columns')
hdf['ratio_predicted16'] = hdf.apply(lambda x: fill_missing(x, 'deaths16', 2016), axis = 'columns')
hdf['ratio_predicted17'] = hdf.apply(lambda x: fill_missing(x, 'deaths17', 2017), axis = 'columns')

In [22]:
hdf[(hdf.ratio_predicted13 + hdf.ratio_predicted14 + hdf.ratio_predicted15 + hdf.ratio_predicted16 + hdf.ratio_predicted17) != hdf.total_deaths]

,county_code,geometry,county,state,deaths13,deaths14,deaths15,deaths16,deaths17,pop13,...,prate_13,prate_14,prate_15,prate_16,prate_17,ratio_predicted13,ratio_predicted14,ratio_predicted15,ratio_predicted16,ratio_predicted17
27,1055,"POLYGON ((-86.10507699999999 34.201546, -85.84...","Etowah County, AL",AL,NaN,NaN,NaN,NaN,15.0,103931,...,199.1,191.4,171.9,156.8,130.5,1.0,2.0,3.0,6.0,15.0
58,1117,"POLYGON ((-86.51584800000001 33.544313, -86.37...","Shelby County, AL",AL,NaN,NaN,NaN,16.0,10.0,204180,...,119.4,110.5,103.4,100.7,89.6,2.0,3.0,4.0,16.0,10.0
68,4003,"POLYGON ((-109.831183 32.427018, -109.113704 3...","Cochise County, AZ",AZ,NaN,NaN,NaN,NaN,NaN,129473,...,83.3,82.7,83.5,79.0,69.0,1.0,2.0,2.0,3.0,7.0
75,4015,"POLYGON ((-112.898269 37.000263, -112.542268 3...","Mohave County, AZ",AZ,NaN,NaN,NaN,NaN,NaN,203030,...,140.5,142.2,133.9,127.5,111.8,1.0,1.0,1.0,2.0,5.0
78,4021,"POLYGON ((-111.578327 33.467636, -111.041587 3...","Pinal County, AZ",AZ,NaN,NaN,NaN,NaN,NaN,389350,...,64.1,65.6,63.2,58.3,50.1,2.0,2.0,3.0,5.0,9.0
141,5119,"POLYGON ((-92.228296 34.957364, -92.118758 35....","Pulaski County, AR",AR,NaN,NaN,NaN,NaN,15.0,391284,...,120.2,117.5,111.6,105.2,98.3,5.0,5.0,7.0,6.0,15.0
153,5143,"POLYGON ((-94.013779 36.206106, -93.887809 36....","Washington County, AR",AR,NaN,NaN,NaN,NaN,NaN,216410,...,108.2,104.4,101.6,98.5,91.9,2.0,2.0,3.0,2.0,4.0
157,6001,"POLYGON ((-122.269314 37.903958, -121.557312 3...","Alameda County, CA",CA,NaN,NaN,NaN,11.0,NaN,1578891,...,39.4,38.0,33.6,31.1,26.4,4.0,4.0,5.0,11.0,9.0
163,6013,"POLYGON ((-121.628512 38.101128, -121.57922 38...","Contra Costa County, CA",CA,NaN,NaN,NaN,NaN,17.0,1094205,...,55.5,51.8,45.8,40.8,36.2,4.0,4.0,5.0,8.0,17.0
166,6019,"POLYGON ((-119.021489 37.586295, -118.775026 3...","Fresno County, CA",CA,NaN,NaN,NaN,NaN,NaN,955272,...,68.1,66.1,59.6,57.8,52.8,3.0,4.0,4.0,7.0,9.0


In [23]:
print(hdf.ratio_predicted13.sum())
print(hdf.ratio_predicted14.sum())
print(hdf.ratio_predicted15.sum())
print(hdf.ratio_predicted16.sum())
print(hdf.ratio_predicted17.sum())

2577.0
4946.0
8905.0
18485.0
26955.0


In [24]:
difference_list = []
for state in hdf.state.unique():
    for year in range(2013, 2018):
        pred_state_sum = hdf[hdf.state == state]['ratio_predicted' + str(year-2000)].sum()
        actual_state_sum = fent_by_state[(fent_by_state.state_abbr == state) & (fent_by_state.year == 2013)].deaths.values[0]
        state_difference = (pred_state_sum - actual_state_sum)**2
        difference_list.append(state_difference)
print('squared sum state error ratio predictor', sum(difference_list))

squared sum state error ratio predictor 54081062.0


In [25]:
hdf['pop13'] = hdf.pop13.apply(int)
hdf['pop14'] = hdf.pop14.apply(int)
hdf['pop15'] = hdf.pop15.apply(int)
hdf['pop16'] = hdf.pop16.apply(int)
hdf['pop17'] = hdf.pop17.apply(int)

In [26]:
hdf['crude_rate13'] = hdf.deaths13.fillna(0) / hdf.pop13 * 100000
hdf['crude_rate14'] = hdf.deaths14.fillna(0) / hdf.pop14 * 100000
hdf['crude_rate15'] = hdf.deaths15.fillna(0) / hdf.pop15 * 100000
hdf['crude_rate16'] = hdf.deaths16.fillna(0) / hdf.pop16 * 100000
hdf['crude_rate17'] = hdf.deaths17.fillna(0) / hdf.pop17 * 100000

In [27]:
hdf['crude_rate_lag13'] = ps.lag_spatial(w, hdf['crude_rate13'])
hdf['crude_rate_lag14'] = ps.lag_spatial(w, hdf['crude_rate14'])
hdf['crude_rate_lag15'] = ps.lag_spatial(w, hdf['crude_rate15'])
hdf['crude_rate_lag16'] = ps.lag_spatial(w, hdf['crude_rate16'])
hdf['crude_rate_lag17'] = ps.lag_spatial(w, hdf['crude_rate17'])

In [29]:
knn = hdf.copy()
knn['prate_lag13'] = ps.lag_spatial(w, knn['prate_13'].fillna(knn['prate_13'].dropna().mean()))
knn['prate_lag14'] = ps.lag_spatial(w, knn['prate_14'].fillna(knn['prate_14'].dropna().mean()))
knn['prate_lag15'] = ps.lag_spatial(w, knn['prate_15'].fillna(knn['prate_15'].dropna().mean()))
knn['prate_lag16'] = ps.lag_spatial(w, knn['prate_16'].fillna(knn['prate_16'].dropna().mean()))
knn['prate_lag17'] = ps.lag_spatial(w, knn['prate_17'].fillna(knn['prate_17'].dropna().mean()))
knn['prate_13'] = knn['prate_13'].fillna(knn['prate_lag13'])
knn['prate_14'] = knn['prate_14'].fillna(knn['prate_lag13'])
knn['prate_15'] = knn['prate_15'].fillna(knn['prate_lag13'])
knn['prate_16'] = knn['prate_16'].fillna(knn['prate_lag13'])
knn['prate_17'] = knn['prate_17'].fillna(knn['prate_lag13'])

In [ ]:
scaled_df = pd.concat([knn.drop(['state'], axis = 'columns'), pd.get_dummies(knn.state)], axis=1)

In [ ]:
scaled_df = scaled_df.drop(['county_code', 'geometry', 'county'], axis = 'columns')

In [ ]:
scaled_df = scaled_df.drop(['ratio_predicted13', 'ratio_predicted14', 'ratio_predicted15', 'ratio_predicted16', 'ratio_predicted17'], axis = 'columns')

In [ ]:
scaled_df.columns

In [ ]:
scaler = StandardScaler()
scaled_df[['deaths13', 'deaths14', 'deaths15', 'deaths16', 'deaths17', 'pop13',
       'pop14', 'pop15', 'pop16', 'pop17', 'total_deaths',
       'percent_in_distress', 'prate_13', 'prate_14', 'prate_15', 'prate_16',
       'prate_17', 'crude_rate13', 'crude_rate14', 'crude_rate15',
       'crude_rate16', 'crude_rate17', 'crude_rate_lag13', 'crude_rate_lag14',
       'crude_rate_lag15', 'crude_rate_lag16', 'crude_rate_lag17',
       'prate_lag13', 'prate_lag14', 'prate_lag15', 'prate_lag16',
       'prate_lag17']] = scaler.fit_transform(knn[['deaths13', 'deaths14', 'deaths15', 'deaths16', 'deaths17', 'pop13',
       'pop14', 'pop15', 'pop16', 'pop17', 'total_deaths',
       'percent_in_distress', 'prate_13', 'prate_14', 'prate_15', 'prate_16',
       'prate_17', 'crude_rate13', 'crude_rate14', 'crude_rate15',
       'crude_rate16', 'crude_rate17', 'crude_rate_lag13', 'crude_rate_lag14',
       'crude_rate_lag15', 'crude_rate_lag16', 'crude_rate_lag17',
       'prate_lag13', 'prate_lag14', 'prate_lag15', 'prate_lag16',
       'prate_lag17']])

In [ ]:
knn_imputed = KNN(k=5, max_value = 9).solve(scaled_df.values, np.isnan(scaled_df.values))

In [ ]:
knn_imputed = pd.DataFrame(knn_imputed, columns = ['deaths13', 'deaths14', 'deaths15', 'deaths16', 'deaths17', 'pop13',
       'pop14', 'pop15', 'pop16', 'pop17', 'total_deaths',
       'percent_in_distress', 'prate_13', 'prate_14', 'prate_15', 'prate_16',
       'prate_17', 'crude_rate13', 'crude_rate14', 'crude_rate15',
       'crude_rate16', 'crude_rate17', 'crude_rate_lag13', 'crude_rate_lag14',
       'crude_rate_lag15', 'crude_rate_lag16', 'crude_rate_lag17',
       'prate_lag13', 'prate_lag14', 'prate_lag15', 'prate_lag16',
       'prate_lag17', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
       'GA', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI',
       'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY',
       'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT',
       'WA', 'WI', 'WV', 'WY'])

In [ ]:
knn_imputed[['deaths13', 'deaths14', 'deaths15', 'deaths16', 'deaths17', 'pop13',
       'pop14', 'pop15', 'pop16', 'pop17', 'total_deaths',
       'percent_in_distress', 'prate_13', 'prate_14', 'prate_15', 'prate_16',
       'prate_17', 'crude_rate13', 'crude_rate14', 'crude_rate15',
       'crude_rate16', 'crude_rate17', 'crude_rate_lag13', 'crude_rate_lag14',
       'crude_rate_lag15', 'crude_rate_lag16', 'crude_rate_lag17',
       'prate_lag13', 'prate_lag14', 'prate_lag15', 'prate_lag16',
       'prate_lag17']] = scaler.inverse_transform(knn_imputed[['deaths13', 'deaths14', 'deaths15', 'deaths16', 'deaths17', 'pop13',
       'pop14', 'pop15', 'pop16', 'pop17', 'total_deaths',
       'percent_in_distress', 'prate_13', 'prate_14', 'prate_15', 'prate_16',
       'prate_17', 'crude_rate13', 'crude_rate14', 'crude_rate15',
       'crude_rate16', 'crude_rate17', 'crude_rate_lag13', 'crude_rate_lag14',
       'crude_rate_lag15', 'crude_rate_lag16', 'crude_rate_lag17',
       'prate_lag13', 'prate_lag14', 'prate_lag15', 'prate_lag16',
       'prate_lag17']])

In [ ]:
knn_imputed

In [28]:
hdf['ratio_pred_rate13'] = hdf['ratio_predicted13'] / hdf.pop13 * 100000
hdf['ratio_pred_rate14'] = hdf['ratio_predicted14'] / hdf.pop14 * 100000
hdf['ratio_pred_rate15'] = hdf['ratio_predicted15'] / hdf.pop15 * 100000
hdf['ratio_pred_rate16'] = hdf['ratio_predicted16'] / hdf.pop16 * 100000
hdf['ratio_pred_rate17'] = hdf['ratio_predicted17'] / hdf.pop17 * 100000

In [29]:
hdf['ratio_pred_rate_lag13'] = ps.lag_spatial(w, hdf['ratio_pred_rate13'])
hdf['ratio_pred_rate_lag14'] = ps.lag_spatial(w, hdf['ratio_pred_rate14'])
hdf['ratio_pred_rate_lag15'] = ps.lag_spatial(w, hdf['ratio_pred_rate15'])
hdf['ratio_pred_rate_lag16'] = ps.lag_spatial(w, hdf['ratio_pred_rate16'])
hdf['ratio_pred_rate_lag17'] = ps.lag_spatial(w, hdf['ratio_pred_rate17'])

In [30]:
hdf.to_csv('ratio_predicted.csv')

In [63]:
import plotly
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
plotly.tools.set_credentials_file(username='steve.riesenberg', api_key='XYKDTo19pHDM2oPU97oc')

# df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
# df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
# df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
# df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']

colorscale = ['#edf8e9',
'#c7e9c0',
'#a1d99b',
'#74c476',
'#31a354',
'#006d2c']

endpts = [0.1, 10, 20, 30, 40]
fips = hdf.county_code.tolist()
values = hdf.ratio_pred_rate17.tolist()

fig = ff.create_choropleth(
    fips = fips, values = values, scope = ['usa'],
    binning_endpoints = endpts, colorscale = colorscale,
    show_state_data = False,
    show_hover = True, centroid_marker = {
        'opacity': 0
    },
    asp = 2.9,
    title = 'Fentanyl Overdose Deaths per 100,000 People',
    legend_title = 'Rate per 100,000'
)
plotly.plotly.iplot(fig, filename = 'choropleth_full_usa')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~steve.riesenberg/0 or inside your plot.ly account where it is named 'choropleth_full_usa'


/home/steve/anaconda3/envs/plotly/lib/python3.7/site-packages/chart_studio/api/v1/clientresp.py:44: UserWarning:

Estimated Draw Time Slow

/home/steve/anaconda3/envs/plotly/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [67]:
hdf[['county', 'pop17', 'deaths17', 'ratio_predicted17', 'ratio_pred_rate17']].sort_values(by = 'ratio_pred_rate17', ascending = False)

,county,pop17,deaths17,ratio_predicted17,ratio_pred_rate17
2933,"Cabell County, WV",94958,118.0,118.0,124.265465
2977,"Wayne County, WV",40153,38.0,38.0,94.638010
2028,"Fayette County, OH",28752,24.0,24.0,83.472454
2061,"Montgomery County, OH",531542,434.0,434.0,81.649239
996,"Gallatin County, KY",8776,NaN,6.0,68.368277
1181,"Baltimore city, MD",611648,398.0,398.0,65.070106
2929,"Berkeley County, WV",114920,73.0,73.0,63.522450
2949,"Lincoln County, WV",20825,13.0,13.0,62.424970
2018,"Clinton County, OH",42009,25.0,25.0,59.511057
750,"Wayne County, IN",66185,39.0,39.0,58.925738
